# Outline

* array of acceptable genres
* Input: 10 games -->

* all movies w/in these genres, w/in accepted
* --> console as filer --> genres as filter, queried data - keywords mlo

* query: {name:keywords}

kelly will do this using postman

* data in: name, keywords

* process: calculate dist as # common keywords, to keywords in games user chose

* output: closest games to target keywords

# Code

In [69]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsRegressor

games = pd.read_csv("IGDB_games.csv", sep=",")
games

,id,category,created_at,external_games,keywords,name,similar_games,slug,tags,updated_at,...,hypes,dlcs,bundles,expansions,expanded_games,version_parent,version_title,remakes,standalone_expansions,remasters
0,73817,0,2017-10-05 00:00:00,"[147971, 1870903]","[1097, 1219, 2396, 4260, 5570]",Spy Snatcher,"[18290, 58702, 58867, 59500, 59720, 79064, 800...",spy-snatcher,"[536872009, 536872131, 536873308, 536875172, 5...",2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,112119,0,2018-11-16 00:00:00,"[1616043, 1911731]",NaN,Woohoojin Bridges,NaN,woohoojin-bridges,NaN,2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008,28387,0,2017-05-02 16:09:39,"[12293, 137034, 281289, 1708343]","[464, 1026]",CABAL Online,"[28309, 30245, 34155, 35994, 55038, 55199, 103...",cabal-online,"[1, 268435460, 268435468, 536871376, 536871938]",2021-04-19 17:09:56,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11009,70300,0,2017-09-28 00:00:00,"[142023, 1874213]",[1166],"Gold, Silver, Bronze","[15851, 109606, 113149, 113895, 116352, 116353...",gold-silver-bronze,"[1, 268435470, 536872078]",2020-08-26 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
class User:
    def __init__(self, gn, g, t):  # self, game_names, genres, themes):
        self.name = "TESTerrr"
        self.game_names = pd.DataFrame(gn, columns=['name'])
        self.genres = pd.DataFrame(g, columns=['name'])
        self.themes = pd.DataFrame(t, columns=['name'])

# '''    def printUser(self):
#         print(self.name, "'s profile:\n- ", self.game_names,
#               "\n- ", self.genres, "\n-", self.themes)'''
    # def getIds(self):



In [63]:
'''USER UPPERCASE AS FIRST LETTER'''
gn, g, t = ["Skyrim","Destiny","Mario Kart","Forza"],["Adventure","Racing","Shooter"],["Action","Survival"]
new_user = User(gn, g, t)


# type(new_user.genres)

In [76]:
# to be getIDs
genre_key = pd.read_csv("../DataExtraction/ids_with_genres.csv", sep=",")
new_user.genres = new_user.genres.merge(genre_key, how="left", on="name")
theme_key = pd.read_csv("../DataExtraction/themes_with_ids.csv", sep=",")
new_user.themes = new_user.themes.merge(theme_key, how="left", on="name")
new_user.themes 


,name,id
0,Action,1
1,Survival,21


<h1> example with fake data </h1>

 EX:
 `{'name': ['Skyrim', 'Destiny', 'Mario Kart', 'Forza'], 'adventure': [1, 0,0,0], 'racing': [0, 0,1,1], 'shooter': [0, 1,0,0]}`


##### just use genre and themes for now, like we talked about

In [83]:
# Features in our model

features = ["genres", "themes"]

X_train_dict = games[features].to_dict(orient="records")
X_new_dict = {
    "genres": new_user.genres["id"][0], # starting with one value
    "themes": new_user.themes["id"][0]
}
# y_train = games["name"]

# Dummy encoding
vec = DictVectorizer(sparse=False)
vec.fit(X_train_dict)
X_train = vec.transform(X_train_dict)
X_new = vec.transform(X_new_dict)

# skip scaling

# K-Nearest Neighbors Model
model = KNeighborsRegressor(n_neighbors=30)
model.fit(X_train_sc, y_train)
model.predict(X_new_sc)

array([[31.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [ ]:
X_train = df[["adventure", "racing", "shooter"]]
y_train = df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [ ]:
x_new = pd.Series(dtype=int)
x_new["adventure"] = 0
x_new["racing"] = 1
x_new["shooter"] = 0

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

In [ ]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

In [ ]:
inds_nearest = dists_sorted.index[:9]
inds_nearest

In [ ]:
df.loc[inds_nearest]

<h1> example with 100 games </h1>
I can't work :(

In [ ]:
# read the csv with all the games
games = pd.read_csv("test_dataset.csv", sep=",")

# split the string into a list of strings
games.genres = games.genres.str.split(', ').to_frame()

# fill in [0] for all the NaN values
games['genres'] = games['genres'].fillna({i: [0] for i in df.index})

games['genres'][97]

In [ ]:
# convert the list of strings into a list of ints
for i, genre_list in enumerate(games.genres):
    for genre in genre_list:
        games.genres[i] = list(map(int, games.genres[i]))

In [ ]:
# Getting distinct genre types for generating columns of genre type.
genre_columns = list(set([j for i in games['genres'].tolist() for j in i]))

In [ ]:
# copy over name and id columns to new df
selected_columns = games[["id","name"]]
genre_df = selected_columns.copy()

In [ ]:
# Iterating over every list to create and fill values into columns.
for j in genre_columns:
    genre_df[j] = 0
for i in range(games.shape[0]):
    for j in genre_columns:
        if(j in games['genres'].iloc[i]):
            genre_df.loc[i,j] = 1

In [ ]:
genre_df

In [ ]:
# Features in our model. All quantitative, except Neighborhood.
features = genre_columns

X_train_dict = genre_df[features].to_dict(orient="records")
y_train = genre_df["name"]

In [ ]:
X_train = genre_df[genre_columns]
y_train = genre_df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [ ]:
x_new = pd.Series(0, index=genre_columns)

x_new[9] = 1
x_new[30] = 1
x_new[31] = 1

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

In [ ]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

In [ ]:
inds_nearest = dists_sorted.index[:9]
inds_nearest

In [ ]:
genre_df.loc[inds_nearest]